In [1]:
# ========================Load data=========================
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from keras.layers import Input, Embedding, Activation, Flatten, Dense
from keras.layers import Conv1D, MaxPooling1D, Dropout
from keras.models import Model
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer

In [2]:
df = pd.read_excel("../dataset.xlsx", header=None)
df = df.drop([0,2,3,5,6,7,8,9,10,11,12,13,14], axis=1)

In [3]:
test_df = df.iloc[1:301,:]
train_df = df.iloc[301:,:]

## Output Target Column

In [4]:
df2 = pd.read_excel('../dataset.xlsx') #Excelden oku

In [5]:
targetColumn = list(df2['TANILAR'])
targetCorpus = []
for i in targetColumn:
    row = i.split(", ")
    for c in row:
        if c not in targetCorpus:
            targetCorpus.append(c)
targetCorpus = np.array(targetCorpus)

In [6]:
y_data = []
for i in targetColumn:
    row = i.split(", ")
    actualRow=[]
    for q in targetCorpus:
        if q not in row:
            actualRow.append(0.0)
        else:
             actualRow.append(1.0)
    y_data.append(actualRow)
y_data = np.array(y_data)
y_test = y_data[:300]
y_train = y_data[300:]

In [7]:
len(train_df),len(y_train)

(1317, 1317)

In [8]:
# convert string to lower case
train_texts = train_df[4].values
train_texts = [s.lower() for s in train_texts]

test_texts = test_df[4].values
test_texts = [s.lower() for s in test_texts]


In [9]:
test_texts[0]

'halsizlik yorgunluk eklem ağrısı üşüme sık ağız yarası'

In [10]:
# =======================Convert string to index================
# Tokenizer
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(train_texts)
# If we already have a character list, then replace the tk.word_index
# If not, just skip below part

In [11]:
# -----------------------Skip part start--------------------------
# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
# -----------------------Skip part end----------------------------

In [12]:
# Convert string to index
train_sequences = tk.texts_to_sequences(train_texts)
test_texts = tk.texts_to_sequences(test_texts)

In [13]:
# Padding
train_data = pad_sequences(train_sequences, maxlen=1014, padding='post')
test_data = pad_sequences(test_texts, maxlen=1014, padding='post')

In [14]:
# Convert to numpy array
train_data = np.array(train_data, dtype='float32')
test_data = np.array(test_data, dtype='float32')

In [16]:
train_data[0]

array([ 9.,  4., 18., ...,  0.,  0.,  0.], dtype=float32)

In [346]:

from keras.utils import to_categorical
train_classes = to_categorical(y_data)

In [347]:


train_classes = y_train
test_classes = y_test



In [348]:
# =====================Char CNN=======================
# parameter
input_size = 1014
vocab_size = len(tk.word_index)
embedding_size = 69
conv_layers = [[256, 7, 3],
               [256, 7, 3],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, -1],
               [256, 3, 3]]

fully_connected_layers = [1024, 1024]
num_of_classes = len(targetCorpus)
dropout_p = 0.5
optimizer = 'adam'
loss = 'categorical_crossentropy'


In [349]:

# Embedding weights
embedding_weights = []  # (70, 69)
embedding_weights.append(np.zeros(vocab_size))  # (0, 69)

for char, i in tk.word_index.items():  # from index 1 to 69
    onehot = np.zeros(vocab_size)
    onehot[i - 1] = 1
    embedding_weights.append(onehot)

embedding_weights = np.array(embedding_weights)
print('Load')

Load


In [350]:

# Embedding layer Initialization
embedding_layer = Embedding(vocab_size + 1,
                            embedding_size,
                            input_length=input_size,
                            weights=[embedding_weights])

# Model Construction
# Input
inputs = Input(shape=(input_size,), name='input', dtype='int64')  # shape=(?, 1014)
# Embedding
x = embedding_layer(inputs)

In [351]:
# Conv
for filter_num, filter_size, pooling_size in conv_layers:
    x = Conv1D(filter_num, filter_size)(x)
    x = Activation('relu')(x)
    if pooling_size != -1:
        x = MaxPooling1D(pool_size=pooling_size)(x)  # Final shape=(None, 34, 256)
x = Flatten()(x)  # (None, 8704)
# Fully connected layers
for dense_size in fully_connected_layers:
    x = Dense(dense_size, activation='relu')(x)  # dense_size == 1024
    x = Dropout(dropout_p)(x)
# Output Layer
predictions = Dense(num_of_classes, activation='softmax')(x)
# Build model
model = Model(inputs=inputs, outputs=predictions)
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])  # Adam, categorical_crossentropy
model.summary()


Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 1014)]            0         
_________________________________________________________________
embedding_4 (Embedding)      (None, 1014, 69)          4830      
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 1008, 256)         123904    
_________________________________________________________________
activation_24 (Activation)   (None, 1008, 256)         0         
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 336, 256)          0         
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 330, 256)          459008    
_________________________________________________________________
activation_25 (Activation)   (None, 330, 256)          0   

In [352]:
# # 1000 training samples and 100 testing samples
# indices = np.arange(train_data.shape[0])
# np.random.shuffle(indices)
#
# x_train = train_data[indices][:1000]
# y_train = train_classes[indices][:1000]
#
# x_test = test_data[:100]
# y_test = test_classes[:100]

indices = np.arange(train_data.shape[0])


In [357]:
len(y_train),len(x_train),len(x_test),len(y_test)

(1317, 1317, 300, 300)

In [358]:

np.random.shuffle(indices)

x_train = train_data[indices]
y_train = train_classes[indices]

x_test = test_data
y_test = test_classes


In [359]:
len(y_train)

1317

In [360]:
# Training
model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=128,
          epochs=10,
          verbose=2)

Epoch 1/10
11/11 - 10s - loss: 126941568.0000 - accuracy: 0.0440 - val_loss: 1204468864.0000 - val_accuracy: 0.0633
Epoch 2/10
11/11 - 10s - loss: 10614368256.0000 - accuracy: 0.0402 - val_loss: 73338994688.0000 - val_accuracy: 0.0500
Epoch 3/10
11/11 - 10s - loss: 369249452032.0000 - accuracy: 0.0289 - val_loss: 2142050713600.0000 - val_accuracy: 0.0633
Epoch 4/10
11/11 - 10s - loss: 7352042389504.0000 - accuracy: 0.0357 - val_loss: 35143488110592.0000 - val_accuracy: 0.1400
Epoch 5/10
11/11 - 10s - loss: 86883740680192.0000 - accuracy: 0.0395 - val_loss: 381739511840768.0000 - val_accuracy: 0.0467
Epoch 6/10
11/11 - 10s - loss: 779528376942592.0000 - accuracy: 0.0342 - val_loss: 3277293048823808.0000 - val_accuracy: 0.0433
Epoch 7/10
11/11 - 10s - loss: 5315163225849856.0000 - accuracy: 0.0486 - val_loss: 21244682067509248.0000 - val_accuracy: 0.1400
Epoch 8/10
11/11 - 11s - loss: 28118671998058496.0000 - accuracy: 0.0364 - val_loss: 111051292880666624.0000 - val_accuracy: 0.0633
Epo